# Clustering Neighborhoods in Toronto

#### Import all the libraries that might be needed

In [1]:
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Kalaha\anaconda2

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py38haa244fe_0         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  conda                                4.9.0-py38h9bdc248_1 --> 4.9.2-py38haa244fe_0




geographiclib-1.50   | 34 KB   

#### Convert the table on the Wikipedia page into a pandas dataframe

In [60]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url)
df = df[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Remove all the Boroughs that are "Not assigned" from the dataframe

In [61]:
df.set_index('Borough', inplace=True)
df.head()

,Postal Code,Neighbourhood
Borough,,
Not assigned,M1A,Not assigned
Not assigned,M2A,Not assigned
North York,M3A,Parkwoods
North York,M4A,Victoria Village
Downtown Toronto,M5A,"Regent Park, Harbourfront"


In [62]:
df.drop(index='Not assigned', inplace=True)
df.reset_index(inplace=True)
df.head()

,Borough,Postal Code,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,"Regent Park, Harbourfront"
3,North York,M6A,"Lawrence Manor, Lawrence Heights"
4,Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government"


#### Check to make sure there are no Neighbourhoods left that are "Not assigned"

In [64]:
df.sort_values('Neighbourhood')

,Borough,Postal Code,Neighbourhood
78,Scarborough,M1S,Agincourt
93,Etobicoke,M8W,"Alderwood, Long Branch"
28,North York,M3H,"Bathurst Manor, Wilson Heights, Downsview North"
39,North York,M2K,Bayview Village
55,North York,M5M,"Bedford Park, Lawrence Manor East"
20,Downtown Toronto,M5E,Berczy Park
58,Scarborough,M1N,"Birch Cliff, Cliffside West"
43,West Toronto,M6K,"Brockton, Parkdale Village, Exhibition Place"
100,East Toronto,M7Y,"Business reply mail Processing Centre, South C..."
87,Downtown Toronto,M5V,"CN Tower, King and Spadina, Railway Lands, Har..."


#### Reaarrange and Rename the columns to match the example given

In [68]:
df = df[['Postal Code', 'Borough', 'Neighbourhood']]
df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Note the shape of the dataframe: 103 Rows, 3 Columns

In [67]:
df.shape

(103, 3)